# Rhumlogng SQLALChemy access

I wanted to see how to use SqlAlchemy on the database in RhumlogNg.

Please note. **You need to take a copy of the database** in case you do something stupid.

Please do something like this...

    cp ~/Documents/dv3a.db  ~/dev/Ham/Db/log.db

## Map an existing database

Sqlalchemy can 'map' / mirror an existing database - please note the connection string with three '/'
s means relative path.... 4 '/' would be an absolute path.

With the Db Mirrored - we can then create a Class which maps to a table. In out case there are only a few tables.

## Rhumlogng Tables

There are 4 tables in the database.

  - prefs
  - dxlist
  - data
  - logbook

### Prefs 

The items you set in the main app seem to be stored here.

### dxlist 

This seems to be a list of the worked Dx, and if the QSO has been confirmed. But it has a field called 'info' which is a formatted string.

### data 

This table appears empty

### logbook

These are all the QSO's. 

In [1]:
#
# Import the libraries for sqlalchemy
#

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import select          #Needed for selecting data not mapping the Db

Base = automap_base()

# point the connection string to the database
CONN_STR = 'sqlite:///log.db'
engine = create_engine(CONN_STR)

# reflect the tables
Base.prepare(autoload_with=engine)

In [2]:

# Now create an Alias to make the code look a little cleaner
# We want to have a look at 2 tables. Logbook and dxlist
Logbook = Base.classes.logbook
Dxlist  = Base.classes.dxlist

session = Session(engine)


In [3]:
# With the Db mapped - we can now generate our select statement.
# We will start with something simple and easy.

stmt = select(Logbook.callsign,Logbook.band).where(Logbook.callsign != "spongebob")
result = session.execute(stmt)
result.all()[:10]    # Just print the first 10 records

[('JG2RFJ', '10m'),
 ('IT9RZU', '10m'),
 ('BG8TFN', '10m'),
 ('JM1VNJ', '10m'),
 ('EA4URE', '10m'),
 ('JF1RYU', '10m'),
 ('VK3IU', '10m'),
 ('DL6RAI', '10m'),
 ('OM8ON', '10m'),
 ('YB1LUE', '10m')]

# Lets have a look at the DxList 


The table looks like this 

   CREATE TABLE dxlist(dxcc VarChar, info VarChar, dxccadif integer, PRIMARY KEY(dxcc)); 

So lets see the Philippines data.

In [4]:
stmt = select(Dxlist.dxcc,Dxlist.info,Dxlist.dxccadif).where(Dxlist.dxcc == "DU")
result = session.execute(stmt)
result.all()

[('DU', 'L.. .L. WXX ... WXL ... LXX ... XXX .W. ... ...', 375)]

This data looks vey familiar

![./DxList.png](./DxList.png)

To to remind myself, what this means 

  - W
    - Worked Not Confirmed 
  - X
    - Worked Confirmed LOTW&Paper 
  - L
    - Worked Confirmed LOTW
   
The order is 

   **CW** **DATA** **PHONE** 


## Improvements ?

To me this is not helpful. As I believe we should have some more tables 

    - Mode
      - CW, DATA, PHONE
    - Status 
      - NOT WORKED
      - NOT CONFIRMED
      - CONFIRMED
    - Band 
      - 180
      - 80 etc

I will create the Database tables outside the ORM - and reflecting and mapping may confuse me !! 


###  MyDefinitions

```sql
DROP TABLE IF EXISTS Band;
DROP TABLE IF EXISTS Mode;
DROP TABLE IF EXISTS Status;
DROP TABLE IF EXISTS BMS;

create table Band (bid Integer, name VarChar, Primary Key (bid DESC));
create table Mode (mid Integer, name VarChar, Primary Key (mid DESC));
create table Status (sid Integer, code VarChar, name VarChar, Primary Key (sid DESC));

CREATE TABLE BMS (bid Integer NOT NULL, mid Integer NOT NULL, sid Integer NOT NULL, dxccadif Integer NOT NULL,
    PRIMARY KEY (bid, mid, sid, dxccadif),
    FOREIGN KEY (bid)REFERENCES Band (bid)
    FOREIGN KEY (mid)REFERENCES Mode (mid)
    FOREIGN KEY (sid)REFERENCES Status (sid)
    FOREIGN KEY (dxccadif) REFERENCES dxlist(dxccadif)
    );
    
insert into Band values (160,"180m");
insert into Band values (80,"80m");
insert into Band values (60,"60m");
insert into Band values (40,"40m");
insert into Band values (30,"30m");
insert into Band values (20,"20m");
insert into Band values (17,"17m");
insert into Band values (15,"15m");
insert into Band values (12,"12m");
insert into Band values (10,"10m");
insert into Band values (6,"6m");
insert into Band values (2,"2m");
insert into Band values (1,"Sat");

insert into Mode values(1,"CW");
insert into Mode values(2,"DATA");
insert into Mode values(3,"PHONE");

insert into Status values (0,'N',"NOT WORKED");
insert into Status values (1,'S',"NOT CONFIRMED");
insert into Status values (2,'X',"CONFIRMED");

```

It is best to place these commands into a file say **my_upgrades.sql** then to run an update statement

    sqlite3 log.db < my_upgrades.sql 

You can then check this by

```
select count(*) from Band;
select count(*) from Mode;
select count(*) from Status;
```

And you should see

    13
     3
     3 


## We have Added the tables - now what ??

We need to effectivly create a Matierialize view (sorry IBM DB2/Oracle stuff).... 
We do not want to be making a complex query all the time - instead we want to look up in our BMS (Band Mode Status) table. 

In order to do that we need to populate it. 

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import select          #Needed for selecting data not mapping the Db
from sqlalchemy.sql import text        #Needed for clearing BMS
Base = automap_base()

# point the connection string to the database
CONN_STR = 'sqlite:///log.db'
engine = create_engine(CONN_STR)

# reflect the tables
Base.prepare(autoload_with=engine)
# Now create an Alias to make the code look a little cleaner
# We want to have a look at 2 tables. Logbook and dxlist
Logbook = Base.classes.logbook
Dxlist  = Base.classes.dxlist
Band    = Base.classes.Band
Mode    = Base.classes.Mode
Status  = Base.classes.Status
BMS     = Base.classes.BMS
session = Session(engine)


# Populate BMS

We need to create a Record for each Band/Mode/DX 

By default we will assume that all BMS are not worked.

In [6]:
bs = session.execute(select(Band.bid)).all()
ms = session.execute(select(Mode.mid)).all()
ss = session.execute(select(Status.sid).where(Status.name == "NOT WORKED")).all()
# We now need the DXCC ID's 
dxi = session.execute(select(Dxlist.dxccadif,Dxlist.dxcc)).all()

In [7]:
# We will create a List of BMS objects - then add using the Session build add.

session.execute(text('DELETE FROM BMS'))
session.commit()
print("BMS was truncated")
ToAdd=[]
for b in bs:
    for m in ms:
        for s in ss:
            for d in dxi:
              ToAdd.append(BMS(bid=b[0],mid=m[0],sid=s[0],dxccadif=d[0]))
print(f"We have {len(ToAdd)} objects to Add.")
session.add_all(ToAdd)
session.commit()

BMS was truncated
We have 15756 objects to Add.


In [13]:
# We will now get all the Sent but not Confirmed (Ignoring for the moment) 
# For Band/Mode/DXCC 
# The SQL is this 
#     select  distinct l.dxccadif, m.mid,l.mode,s.sid,s.name , l.lotwqsl, l.band  from logbook l ,Mode m, Status s, Band b  where m.name=l.mode and b.name=l.band and s.code = l.lotwqsl and s.name="NOT CONFIRMED";
stmt = select(Logbook.dxccadif, Logbook.band, Logbook.mode,Status.name,Logbook.lotwqsl, Status.sid, Mode.mid).where(Mode.name == Logbook.mode and Band.name == Logbook.band and Status.code == Logbook.lotqsql and Status.name =="NOT CONFIRMED" )
non_confirmed_result = session.execute(stmt).all()
print(f"We have {len(non_confirmed_result)} Non Confirmed in Total")

#Sqlalchemy does not appear to like the .distinct(fld,fld) clause.... so we 
#will make these records unique using python
#Cast data to a set and then back to a list

unique_non_confirmed = list(set(non_confirmed_result))
print(f"We have {len(unique_non_confirmed)} Unique Non Confirmed records")

We have 10386 Non Confirmed in Total
We have 1473 Unique Non Confirmed records


/var/folders/ds/nkj_bsw113d71_zp8k3rt4t80000gn/T/ipykernel_1218/2060111363.py:6: SAWarning: SELECT statement has a cartesian product between FROM element(s) "Mode", "logbook" and FROM element "Status".  Apply join condition(s) between each element to resolve.
  non_confirmed_result = session.execute(stmt).all()


In [14]:
unique_non_confirmed


[(293, '10m', 'CW', 'NOT WORKED', 'S', 0, 1),
 (324, '40m', 'CW', 'NOT CONFIRMED', 'S', 1, 1),
 (212, '15m', 'CW', 'CONFIRMED', 'S', 2, 1),
 (227, '20m', 'CW', 'NOT CONFIRMED', 'S', 1, 1),
 (15, '10m', 'CW', 'CONFIRMED', 'S', 2, 1),
 (137, '10m', 'CW', 'CONFIRMED', 'S', 2, 1),
 (499, '15m', 'CW', 'NOT WORKED', 'S', 0, 1),
 (137, '15m', 'CW', 'NOT CONFIRMED', 'X', 1, 1),
 (189, '10m', 'CW', 'CONFIRMED', 'S', 2, 1),
 (327, '10m', 'CW', 'NOT WORKED', 'X', 0, 1),
 (27, '160m', 'CW', 'NOT WORKED', 'X', 0, 1),
 (112, '10m', 'CW', 'NOT CONFIRMED', 'S', 1, 1),
 (225, '10m', 'CW', 'NOT WORKED', 'S', 0, 1),
 (387, '10m', 'CW', 'NOT CONFIRMED', 'S', 1, 1),
 (15, '20m', 'CW', 'NOT WORKED', 'X', 0, 1),
 (176, '15m', 'CW', 'NOT WORKED', 'S', 0, 1),
 (110, '20m', 'CW', 'NOT CONFIRMED', 'S', 1, 1),
 (145, '10m', 'CW', 'NOT WORKED', 'X', 0, 1),
 (339, '80m', 'CW', 'CONFIRMED', 'X', 2, 1),
 (248, '20m', 'CW', 'NOT WORKED', 'S', 0, 1),
 (291, '40m', 'CW', 'CONFIRMED', 'X', 2, 1),
 (110, '10m', 'CW', 'NOT

In [9]:
list(set(non_confirmed_result.all()))

AttributeError: 'list' object has no attribute 'all'